## Dipolar Wakefield simulation of the LHC unshielded bellows with `wakis`

* Creation of the geometry from simple geometry blocks (CSG) -> fully parametrized
* Simulation of the Dipolar X impedance and wake
  
| Wake | Impedance |
| --- | ---------- |
|     |            | 

In [1]:
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt

from wakis import SolverFIT3D
from wakis import GridFIT3D 
from wakis import WakeSolver

%matplotlib ipympl

### Geometry generation using Constructive Solid Geometry (CSG)

In [2]:
# beam pipe
r_pipe = 24e-3      # pipe radius [m]
l_pipe = 320e-3     # pipe length [m]
pipe = pv.Cylinder(center=(0,0,0), direction=(0,0,1), radius=r_pipe, height=l_pipe)

# convolutions
r_conv = 30e-3  # convolution radius [m]
l_conv = 4e-3  # length of each convolution [m]
n_conv = 15      # number of convolutions
l_between_conv = 4e-3  # length between convolutions [m]
z_start = n_conv//2*(l_conv + l_between_conv) - l_conv # start of the convolutions [m]
convolutions = []

for n in range(n_conv):
    z_start_n_conv = -z_start+n*(l_conv+l_between_conv)
    conv = pv.Cylinder(center=(0,0,z_start_n_conv),     # center of the convolution
                       direction=(0,0,1),  # z-direction
                       radius=r_conv, 
                       height=l_conv)
    convolutions.append(conv)  # append to list

# Sum the generated geometry
pipe = pipe.triangulate()                          # triangulate pipe
convolutions = np.sum(convolutions).triangulate()  # triangulate convolutions
bellow = pipe + convolutions  # union of meshes without internal faces

# Save it to file
stl_file = 'notebooks/data/006_LHC_Bellow_generated.stl'
bellow.save(stl_file) #save in [m]
# bellow.scale(1e3).save(stl_file) #save in [mm]

# Plot the generated geometry
bellow.clear_data()   
#bellow.plot(opacity=0.7)

### Domain setup and grid generation

In [3]:
# ---------- Domain setup ---------
# Set up geometry & materials dictionaries
stl_solids = {'bellow': stl_file}
stl_materials = {'bellow': 'vacuum'}

# Domain bounds
xmin, xmax, ymin, ymax, zmin, zmax = bellow.bounds

# Number of mesh cells
Nx = 60
Ny = 60
Nz = 220
print(f'Total number of mesh cells: {Nx*Ny*Nz}')

# set grid and geometry
grid = GridFIT3D(xmin, xmax, ymin, ymax, zmin, zmax, Nx, Ny, Nz, 
                stl_solids=stl_solids, 
                stl_materials=stl_materials,
                use_mesh_refinement=True,
                refinement_tol=1e-8,)

Total number of mesh cells: 792000
Generating grid...
Applying mesh refinement...
* Calculating snappy points...
* Refining x axis with 5 snaps...
Both `ftol` and `xtol` termination conditions are satisfied.
Function evaluations 25, initial cost 3.8333e-04, final cost 1.8979e-05, first-order optimality 5.88e-04.
* Refining y axis with 5 snaps...
Both `ftol` and `xtol` termination conditions are satisfied.
Function evaluations 25, initial cost 3.8333e-04, final cost 1.8979e-05, first-order optimality 5.88e-04.
* Refining z axis with 32 snaps...
`ftol` termination condition is satisfied.
Function evaluations 4293, initial cost 1.0298e-04, final cost 3.3198e-05, first-order optimality 1.06e-06.
Refined grid: Nx = 59, Ny =59, Nz = 219
Importing STL solids...


In [ ]:
grid.inspect()

Widget(value='<iframe src="http://localhost:37263/index.html?ui=P_0x78ec00708990_0&reconnect=auto" class="pyvi…

Exception raised
ConnectionResetError('Cannot write to closing transport')
Traceback (most recent call last):
  File "/home/edelafue/miniconda3/envs/wakis-env/lib/python3.11/site-packages/wslink/protocol.py", line 317, in onCompleteMessage
    await self.sendWrappedMessage(
  File "/home/edelafue/miniconda3/envs/wakis-env/lib/python3.11/site-packages/wslink/protocol.py", line 418, in sendWrappedMessage
    await ws.send_bytes(chunk)
  File "/home/edelafue/miniconda3/envs/wakis-env/lib/python3.11/site-packages/aiohttp/web_ws.py", line 342, in send_bytes
    await self._writer.send(data, binary=True, compress=compress)
  File "/home/edelafue/miniconda3/envs/wakis-env/lib/python3.11/site-packages/aiohttp/http_websocket.py", line 727, in send
    await self._send_frame(message, WSMsgType.BINARY, compress)
  File "/home/edelafue/miniconda3/envs/wakis-env/lib/python3.11/site-packages/aiohttp/http_websocket.py", line 682, in _send_frame
    self._write(header + message)
  File "/home/edelafue

### Boundary conditions and EM solver

In [5]:
# boundary conditions
bc_low=['pec', 'pec', 'pml']
bc_high=['pec', 'pec', 'pml']

n_pml = 10 # number of PML layers
solver = SolverFIT3D(grid, 
                     bc_low=bc_low, bc_high=bc_high, 
                     use_stl=True, bg='pec',
                     n_pml=n_pml)

Assembling operator matrices...
Applying boundary conditions...
Adding material tensors...
Filling PML sigmas...
Calculating maximal stable timestep...
Pre-computing...
Total initialization time: 0.8114991188049316 s


### Wakefield settings & Run

The wakefield simulation will run up to 10 m which will give a partially decayed wake and a non-converged impedance - this will be the starting point of IDDEFIX extrapolation !

In [ ]:
# ------------ Beam source ----------------
# Beam parameters
sigmaz = 10e-3      #[m] -> 2 GHz
q = 1e-9            #[C]
beta = 1.0          # beam beta 
xs = 5e-3           # x source position [m]
ys = 0.             # y source position [m]
xt = 0.             # x test position [m]
yt = 0.             # y test position [m]
# [DEFAULT] tinj = 8.53*sigmaz/c_light  # injection time offset [s] 

# ----------- Solver  setup  ----------
# Wakefield post-processor
wakelength = 10. # [m] -> Partially decayed
skip_cells = n_pml + 2  # no. cells to skip from wake integration (>= PML cells)

results_folder = 'notebooks/006_results/'
wake = WakeSolver(q=q, sigmaz=sigmaz, beta=beta,
                xsource=xs, ysource=ys, xtest=xt, ytest=yt,
                skip_cells=skip_cells, wakelength=wakelength,
                results_folder=results_folder,
                Ez_file=results_folder+'Ez.h5',)

In [ ]:
# Plot settings
import os
if not os.path.exists(results_folder+'img/'): os.mkdir(results_folder+'img/')
plotkw = {'title':results_folder+'img/Ez', 
            'add_patch':'bellow', 'patch_alpha':0.3,
            'vmin':-1e4, 'vmax':1e4,
            'plane': [int(Nx/2), slice(0, Ny), slice(0, Nz)],}

solver.wakesolve(wakelength=wakelength, 
                 wake=wake, 
                 plot=False,
                 plot_every=50, 
                 plot_until=5000,
                 **plotkw)

In [ ]:
solver.wakesolve(wakelength=wakelength, 
                 wake=wake, 
                 plot=False)

Running electromagnetic time-domain simulation...


 55%|█████▌    | 19492/35138 [15:06<11:00, 23.68it/s] 

Or, if results have already been generated:

In [ ]:
wake.load_results(results_folder)

# Modify the number of cells skip from the boundary during integration
# to aboid boundary artifacts -useful in low impedance devices!
wake.skip_cells = 30
wake.solve()

It's done! Now we can plot the results:

In [ ]:
# Plot longitudinal wake potential and impedance
fig1, ax = plt.subplots(1,2, figsize=[12,4], dpi=150)
ax[0].plot(wake.s*1e2, wake.WP, c='tab:red', lw=1.5, label='Wakis')
ax[0].set_xlabel('s [cm]')
ax[0].set_ylabel('Longitudinal wake potential [V/pC]', color='tab:red')
ax[0].legend()
ax[0].set_xlim(xmax=wakelength*1e2)

ax[1].plot(wake.f*1e-9, np.abs(wake.Z), c='tab:blue', alpha=0.8, lw=2, label='Abs')
ax[1].plot(wake.f*1e-9, np.real(wake.Z), ls='--', c='tab:blue', lw=1.5, label='Real')
ax[1].plot(wake.f*1e-9, np.imag(wake.Z), ls=':', c='tab:blue', lw=1.5, label='Imag')
ax[1].set_xlabel('f [GHz]')
ax[1].set_ylabel('Longitudinal impedance [Abs][$\Omega$]', color='tab:blue')
ax[1].legend()

fig1.tight_layout()
fig1.savefig(results_folder+'longitudinal.png')
#plt.show()

In [ ]:
# Plot transverse x wake potential and impedance
fig2, ax = plt.subplots(1,2, figsize=[12,4], dpi=150)
ax[0].plot(wake.s*1e2, wake.WPx, c='tab:orange', lw=1.5, label='Wakis')
ax[0].set_xlabel('s [cm]')
ax[0].set_ylabel('Transverse wake potential X [V/pC]', color='tab:orange')
ax[0].legend()
ax[0].set_xlim(xmax=wakelength*1e2)

ax[1].plot(wake.f*1e-9, np.abs(wake.Zx), c='tab:green', lw=2, label='Abs')
ax[1].plot(wake.f*1e-9, np.real(wake.Zx), c='tab:green', ls='--', lw=1.5, label='Real')
ax[1].plot(wake.f*1e-9, np.imag(wake.Zx), c='tab:green', ls=':', lw=1.5, label='Imag')
ax[1].set_xlabel('f [GHz]')
ax[1].set_ylabel('Transverse impedance X [Abs][$\Omega$]', color='tab:green')
ax[1].legend()

fig2.tight_layout()
fig2.savefig(results_folder+'transverse_x.png')
#plt.show()

In [ ]:
# Plot transverse y wake potential and impedance
fig3, ax = plt.subplots(1,2, figsize=[12,4], dpi=150)
ax[0].plot(wake.s*1e2, wake.WPy, c='tab:brown', lw=1.5, label='Wakis')
ax[0].set_xlabel('s [cm]')
ax[0].set_ylabel('Transverse wake potential Y [V/pC]', color='tab:brown')
ax[0].legend()
ax[0].set_xlim(xmax=wakelength*1e2)

ax[1].plot(wake.f*1e-9, np.abs(wake.Zy), c='tab:pink', lw=2, label='Abs')
ax[1].plot(wake.f*1e-9, np.real(wake.Zy), c='tab:pink', ls='--', lw=1.5, label='Real')
ax[1].plot(wake.f*1e-9, np.imag(wake.Zy), c='tab:pink', ls=':', lw=1.5, label='Imag')
ax[1].set_xlabel('f [GHz]')
ax[1].set_ylabel('Transverse impedance Y [Abs][$\Omega$]', color='tab:pink')
ax[1].legend()

fig3.tight_layout()
fig3.savefig(results_folder+'transverse_y.png')
#plt.show()